# Multi Tool path Gen

In [1]:
import gmsh
import numpy as np

def create_mesh(lc=0.1, r_outer=1.5, export_geo=True):
    gmsh.initialize()
    gmsh.model.add("diamond_to_circle")

    geo_content = "// Gmsh geometry file\n"

    # Define the center of the geometry as Point(0)
    center_tag = gmsh.model.geo.addPoint(0, 0, 0, lc)
    geo_content += f"Point({center_tag}) = {{0, 0, 0, {lc}}};\n"

    # Define the central diamond (square rotated 45 degrees)
    l = 1.0  # Half-diagonal length of the diamond
    diamond_points = []
    coords = [(0, l), (l, 0), (0, -l), (-l, 0)]  # Store actual coordinates
    for x, y in coords:
        tag = gmsh.model.geo.addPoint(x, y, 0, lc)
        diamond_points.append(tag)
        geo_content += f"Point({tag}) = {{{x}, {y}, 0, {lc}}};\n"

    # Define radial extension points
    radial_points = []
    for x, y in coords:
        dx, dy = x / l, y / l  # Unit direction
        tag = gmsh.model.geo.addPoint(x + dx, y + dy, 0, lc)
        radial_points.append(tag)
        geo_content += f"Point({tag}) = {{{x + dx}, {y + dy}, 0, {lc}}};\n"

    # Define outer arc points
    arc_points = []
    for i in range(4):
        angle = i * np.pi / 2
        x, y = r_outer * np.cos(angle), r_outer * np.sin(angle)
        tag = gmsh.model.geo.addPoint(x, y, 0, lc)
        arc_points.append(tag)
        geo_content += f"Point({tag}) = {{{x}, {y}, 0, {lc}}};\n"

    # Create arcs for outer boundary using Point(0) as center
    #outer_arcs = []
    #for i in range(4):
    #    point1, point2 = arc_points[i - 1], arc_points[i]
    #    tag = gmsh.model.geo.addCircleArc(point1, center_tag, point2)
    #    outer_arcs.append(tag)
    #    geo_content += f"Circle({tag}) = {{{point1}, {center_tag}, {point2}}};\n"

    # Create lines for the inner diamond
    diamond_lines = []
    for i in range(4):
        tag = gmsh.model.geo.addLine(diamond_points[i], diamond_points[(i + 1) % 4])
        diamond_lines.append(tag)
        geo_content += f"Line({tag}) = {{{diamond_points[i]}, {diamond_points[(i + 1) % 4]}}};\n"

    # Create radial extension lines
    radial_lines = []
    for i in range(4):
        tag = gmsh.model.geo.addLine(diamond_points[i], radial_points[i])
        radial_lines.append(tag)
        geo_content += f"Line({tag}) = {{{diamond_points[i]}, {radial_points[i]}}};\n"

    # ❌ REMOVE extra arc connections to avoid duplication
    # Only create an arc if it does not already exist in `outer_arcs`
    arc_lines = []
    for i in range(4):
        if radial_points[i] != arc_points[i]:  # Ensure distinct points
            tag = gmsh.model.geo.addCircleArc(radial_points[i], center_tag, arc_points[i])
            arc_lines.append(tag)
            geo_content += f"Circle({tag}) = {{{radial_points[i]}, {center_tag}, {arc_points[i]}}};\n"

    # Create loop for the central square
    square_loop = gmsh.model.geo.addCurveLoop(diamond_lines)
    geo_content += f"Curve Loop({square_loop}) = {{{', '.join(map(str, diamond_lines))}}};\n"

    # Create loops for the transition regions
    transition_loops = []
    for i in range(4):
        tag = gmsh.model.geo.addCurveLoop([
            diamond_lines[i], radial_lines[(i + 1) % 4], arc_lines[i], -radial_lines[i]
        ])
        transition_loops.append(tag)
        geo_content += f"Curve Loop({tag}) = {{{diamond_lines[i]}, {radial_lines[(i + 1) % 4]}, {arc_lines[i]}, {-radial_lines[i]}}};\n"

    # Create loops for outer circular region
    #outer_loop = gmsh.model.geo.addCurveLoop(outer_arcs)
    #geo_content += f"Curve Loop({outer_loop}) = {{{', '.join(map(str, outer_arcs))}}};\n"

    # Create surfaces
    square_surface = gmsh.model.geo.addPlaneSurface([square_loop])
    transition_surfaces = [gmsh.model.geo.addPlaneSurface([loop]) for loop in transition_loops]
    #outer_surface = gmsh.model.geo.addPlaneSurface([outer_loop])

    geo_content += f"Plane Surface({square_surface}) = {{{square_loop}}};\n"
    for i, loop in enumerate(transition_loops):
        geo_content += f"Plane Surface({transition_surfaces[i]}) = {{{loop}}};\n"
    #geo_content += f"Plane Surface({outer_surface}) = {{{outer_loop}}};\n"

    # Synchronize and generate the mesh with structured elements
    #gmsh.model.geo.synchronize()
    #for curve in diamond_lines + radial_lines + outer_arcs:  # Removed arc_lines
    #    gmsh.model.mesh.setTransfiniteCurve(curve, 20)

    #for surface in [square_surface] + transition_surfaces + [outer_surface]:
    #    gmsh.model.mesh.setTransfiniteSurface(surface)
    #    gmsh.model.mesh.setRecombine(2, surface)  # Convert to quads

    gmsh.model.mesh.generate(2)

    # Save mesh and geometry files
    gmsh.write("diamond_to_circle.msh")
    if export_geo:
        with open("diamond_to_circle.geo", "w") as f:
            f.write(geo_content)

    gmsh.finalize()

# Run the mesher and export .geo
create_mesh()


# Run the mesher and export .geo

In [ ]:
create_mesh()